In [1]:
import pandas as pd
from pathlib import Path

# caminho do arquivo
file_path = Path("../data/db_case_sales_ops.xlsx")

# carregar todas as abas
lead = pd.read_excel(file_path, sheet_name="lead")
visitas = pd.read_excel(file_path, sheet_name="visitas")
contratos = pd.read_excel(file_path, sheet_name="contratos")
geral = pd.read_excel(file_path, sheet_name="geral")

# padronizar datas em todas as tabelas
def padroniza_datas(df):
    for col in df.columns:
        col_str = str(col).lower()  
        if "data" in col_str:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


lead = padroniza_datas(lead)
visitas = padroniza_datas(visitas)
contratos = padroniza_datas(contratos)
geral = padroniza_datas(geral)


In [21]:
# ===============================
# LIMPEZA FINAL DA PLANILHA GERAL
# ===============================

# 1. Remover linhas totalmente vazias
geral = geral.dropna(how="all")

# 2. Remover colunas totalmente vazias (se houver)
geral = geral.dropna(axis=1, how="all")

# 3. Derreter (melt) para formato longo
geral_long = geral.melt(
    id_vars="metrica",
    var_name="mes",
    value_name="valor"
)

# 4. Remover linhas sem mês ou sem métrica
geral_long = geral_long.dropna(subset=["mes", "metrica"])

# 5. Converter mes para datetime
geral_long["mes"] = pd.to_datetime(geral_long["mes"], format="%Y-%m")

# 6. Padronizar nome das métricas
geral_long["metrica"] = (
    geral_long["metrica"]
    .str.lower()
    .str.strip()
    .replace({
        "atingimento": "atingimento_vendas",
        "vendas": "vendas",
        "visitas": "visitas",
        "meta": "meta_vendas"
    })
)

# 7. Converter valores para numérico
geral_long["valor"] = pd.to_numeric(geral_long["valor"], errors="coerce")

geral_long.head()


,metrica,mes,valor
0,metas,2024-04-01,NaN
1,vendas,2024-04-01,625.0
2,visitas realizadas,2024-04-01,875.0
3,realizado,2024-04-01,NaN
4,vendas,2024-04-01,548.0


In [23]:
# ===============================================================
# LIMPEZA ESPECÍFICA PARA A PLANILHA DE LEADS
# ===============================================================

lead = lead.rename(columns={
    "id": "lead_id",
    "data_criacao": "data_criacao",
    "data_perda": "data_perda",
    "data_venda": "data_venda",
    "utm_source": "utm_source",
    "sdr": "sdr_responsavel",
    "closer": "closer_responsavel",
    "profissao": "profissao",
    "indicado": "indicado",
    "tipo": "tipo_lead",
    "motivoPerda": "motivo_perda"
})

# ------------------------
# 1. Converter datas
# ------------------------

date_cols = ["data_criacao", "data_perda", "data_venda"]

for col in date_cols:
    lead[col] = pd.to_datetime(lead[col], errors="coerce")

# ------------------------
# 2. Padronizar texto
# ------------------------

str_cols = ["utm_source", "sdr_responsavel", "closer_responsavel",
            "profissao", "tipo_lead", "motivo_perda"]

for col in str_cols:
    lead[col] = lead[col].astype(str).str.lower().str.strip().replace("nan", None)

# ------------------------
# 3. Padronizar coluna indicado_membro
# ------------------------

lead["indicado_membro"] = (
    lead["indicado_membro"]
        .astype(str)
        .str.lower()
        .str.strip()
        .replace({
            "sim": True,
            "yes": True,
            "1": True,
            "true": True,

            "não": False,
            "nao": False,
            "no": False,
            "0": False,
            "false": False,
            "": False,
            "nan": False
        })
)

# garantir tipo booleano
lead["indicado_membro"] = lead["indicado_membro"].astype(bool)

# ------------------------
# 4. Remover duplicados
# ------------------------

leads = lead.drop_duplicates(subset="lead_id", keep="first")

# ------------------------
# 5. Criar coluna status geral do lead
# ------------------------

lead["status_final"] = lead.apply(
    lambda row: "perdido" if pd.notnull(row["data_perda"])
    else "vendido" if pd.notnull(row["data_venda"])
    else "ativo",
    axis=1
)

# preview
leads.head()


C:\Users\milen\AppData\Local\Temp\ipykernel_12508\540264730.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({


,lead_id,data_criacao,data_perda,data_venda,utm_source,sdr_responsavel,closer_responsavel,profissao,indicado_membro,tipo_lead,motivo_perda,status_final
0,79342,2024-04-01,2024-07-24,2024-04-18,typeform-gentileza,none,raquel,psicologia,True,qualificado,none,perdido
1,79305,2024-04-01,2024-09-11,2024-04-04,desconhecido,none,raquel,nutrição,True,qualificado,none,perdido
2,79271,2024-04-01,2024-07-26,2024-04-02,instagram,none,bianca,psicologia,True,qualificado,none,perdido
3,79248,2024-04-01,2024-12-06,2024-04-01,google,none,bianca,odontologia,True,qualificado,none,perdido
4,79226,2024-04-01,2024-11-25,2024-04-30,typeform-blip,none,adriele,outros,True,qualificado,none,perdido


Limpeza da tabela contratos

In [17]:
visitas.columns.tolist()

['id', 'data_visita', 'tipoAgendamento', 'ferramentaAgendamento', 'unidade']

In [10]:
# renomear colunas
contratos = contratos.rename(columns={
    "id": "contrato_id",
    "dataVenda": "data_venda",
    "modalidadeContrato": "modalidade_contrato"
})

# converter datas
contratos["data_venda"] = pd.to_datetime(contratos["data_venda"], errors="coerce")

# remover duplicados
contratos = contratos.drop_duplicates(subset="contrato_id", keep="first")

# remover contratos sem data
contratos = contratos[contratos["data_venda"].notna()]

# padronizar textos
contratos["modalidade_contrato"] = contratos["modalidade_contrato"].str.lower().str.strip()
contratos["plano"] = contratos["plano"].str.lower().str.strip()


In [19]:
# ===============================================================
# LIMPEZA ESPECÍFICA PARA A PLANILHA DE VISITAS
# ===============================================================

# Renomear colunas para formato padronizado
visitas = visitas.rename(columns={
    "id": "id_visita",
    "data_visita": "data_visita",
    "tipoAgendamento": "tipo_agendamento",
    "ferramentaAgendamento": "ferramenta_agendamento",
    "unidade": "unidade"
})

# ------------------------
# 1. Converter datas
# ------------------------

visitas["data_visita"] = pd.to_datetime(visitas["data_visita"], errors="coerce")

# ------------------------
# 2. Padronizar colunas de texto
# ------------------------

str_cols = ["tipo_agendamento", "ferramenta_agendamento", "unidade"]

for col in str_cols:
    visitas[col] = (
        visitas[col]
            .astype(str)
            .str.lower()
            .str.strip()
            .replace("nan", None)
    )

# ------------------------
# 3. Remover duplicados
# ------------------------

visitas = visitas.drop_duplicates(subset=["id_visita"], keep="first")

# ------------------------
# 4. Remover visitas sem data (não agregam ao funil)
# ------------------------

visitas = visitas[visitas["data_visita"].notna()]

# preview final
visitas.head()


,id_visita,data_visita,tipo_agendamento,ferramenta_agendamento,unidade,mes
0,79256,2024-04-01,visita agendada,retool,livance - alphaville,2024-04
1,64143,2024-04-01,visita agendada,retool,livance - alphaville,2024-04
2,79244,2024-04-01,visita agendada,retool,livance - angélica,2024-04
3,78370,2024-04-01,visita agendada,retool,livance - barra da tijuca,2024-04
4,46815,2024-04-01,visita agendada,retool,livance - barra da tijuca,2024-04


In [ ]:
from pathlib import Path

# Criar pasta processed caso não exista
Path("../processed").mkdir(exist_ok=True)

# Salvar CSVs limpos
visitas.to_csv("../processed/visitas_limpo.csv", index=False, encoding="utf-8")
contratos.to_csv("../processed/contratos_limpo.csv", index=False, encoding="utf-8")
geral.to_csv("../processed/geral_limpo.csv", index=False, encoding="utf-8")

print("Arquivos salvos com sucesso na pasta 'processed'!")


Arquivos salvos com sucesso na pasta 'processed'!


In [24]:
Path("../processed").mkdir(exist_ok=True)

lead.to_csv("../processed/leads_limpo.csv", index=False, encoding="utf-8")
